# Creating an RL Based ABR Streaming Algorithm 

In [1]:
import sys
import os
import argparse

import sabre
import numpy as np

## Train and save several models

In [2]:
from emulator import Emulator
from environment import ABR_Env

movie_filename   = '../data/bbb.json'
network_filename = '../data/hd_fs/trace0000.json'

emulator = Emulator(movie_filename, network_filename)

qoe_alpha, qoe_beta, qoe_delta = (1, 1, 1)

env = ABR_Env(
    emulator=emulator, 
    r_multipliers=[qoe_alpha, qoe_beta, qoe_delta]
)
env_train, _ = env.get_sb_env()

In [6]:
from stable_baselines3 import A2C

model_a2c = A2C('MultiInputPolicy', env_train, verbose=1, device='cpu')
model_a2c.learn(total_timesteps=25000)
model_a2c.save('../models/a2c.model')

Using cpu device
-------------------------------------
| time/                 |           |
|    fps                | 181       |
|    iterations         | 100       |
|    time_elapsed       | 2         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.45     |
|    explained_variance | -1.29e-05 |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -6.79e+03 |
|    value_loss         | 8.11e+07  |
-------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 179      |
|    iterations         | 200      |
|    time_elapsed       | 5        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -1.77    |
|    explained_variance | 0.739    |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | -3.18    |
|    va

## Test the ABR Algorithm 

The argparse arguments were copied and pased from the modified `sabre.py` file. Note, minor changes were made where any variable that was used had to be changed to sabre.var_name. 

In [9]:
parser = argparse.ArgumentParser(description = 'Simulate an ABR session.',
                                 formatter_class = argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-n', '--network', metavar = 'NETWORK', default = 'network.json',
                    help = 'Specify the .json file describing the network trace.')
parser.add_argument('-nm', '--network-multiplier', metavar = 'MULTIPLIER',
                    type = float, default = 1,
                    help = 'Multiply throughput by MULTIPLIER.')
parser.add_argument('-m', '--movie', metavar = 'MOVIE', default = 'movie.json',
                    help = 'Specify the .json file describing the movie chunks.')
parser.add_argument('-ml', '--movie-length', metavar = 'LEN', type = float, default = None,
                    help = 'Specify the movie length in seconds (use MOVIE length if None).')
parser.add_argument('-a', '--abr', metavar = 'ABR',
                    default = sabre.abr_default,
                    help = 'Choose ABR algorithm from predefined list (%s), or specify .py module to import.' % ', '.join(sabre.abr_list.keys()))
parser.add_argument('-ab', '--abr-basic', action = 'store_true',
                    help = 'Set ABR to BASIC (ABR strategy dependant).')
parser.add_argument('-ao', '--abr-osc', action = 'store_true',
                    help = 'Set ABR to minimize oscillations.')
parser.add_argument('-gp', '--gamma-p', metavar = 'GAMMAP', type = float, default = 5,
                    help = 'Specify the (gamma p) product in seconds.')
parser.add_argument('-noibr', '--no-insufficient-buffer-rule', action = 'store_true',
                    help = 'Disable Insufficient Buffer Rule.')
parser.add_argument('-ma', '--moving-average', metavar = 'AVERAGE',
                    choices = sabre.average_list.keys(), default = sabre.average_default,
                    help = 'Specify the moving average strategy (%s).' %
                    ', '.join(sabre.average_list.keys()))
parser.add_argument('-ws', '--window-size', metavar = 'WINDOW_SIZE',
                    nargs = '+', type = int, default = [3],
                    help = 'Specify sliding window size.')
parser.add_argument('-hl', '--half-life', metavar = 'HALF_LIFE',
                    nargs = '+', type = float, default = [3, 8],
                    help = 'Specify EWMA half life.')
parser.add_argument('-s', '--seek', nargs = 2, metavar = ('WHEN', 'SEEK'),
                    type = float, default = None,
                    help = 'Specify when to seek in seconds and where to seek in seconds.')
choices = ['none', 'left', 'right']
parser.add_argument('-r', '--replace', metavar = 'REPLACEMENT',
                    #choices = choices,
                    default  =  'none',
                    help = 'Set replacement strategy from predefined list (%s), or specify .py module to import.' % ', '.join(choices))
parser.add_argument('-b', '--max-buffer', metavar = 'MAXBUFFER', type = float, default = 25,
                    help = 'Specify the maximum buffer size in seconds.')
parser.add_argument('-noa', '--no-abandon', action = 'store_true',
                    help = 'Disable abandonment.')
parser.add_argument('-rmp', '--rampup-threshold', metavar = 'THRESHOLD',
                    type = int, default = None,
                    help = 'Specify at what quality index we are ramped up (None matches network).')
parser.add_argument('-v', '--verbose', action = 'store_true',
                    help = 'Run in verbose mode.')

AttributeError: module 'argparse' has no attribute 'Arguments'

In [ ]:
# note: the file paths must be relative to the sabre.py file, not this one
abr_filename     = 'ABR_Zero.py'
movie_filename   = '../data/bbb.json'
network_filename = '../data/hd_fs/trace0000.json'

# parse the args for sabre to run. This would normally be done via cli, but it is convienient here to do in the notebook
args = parser.parse_args([
    '--abr', abr_filename,
    '--movie', movie_filename,
    '--network', network_filename
])
sabre.main(args)